In [1]:
from tqdm.notebook import trange
import chipwhisperer as cw
import os
import numpy as np
import time
import csv

In [2]:
PLATFORM = 'CW308_STM32F3'
CRYPTO_TARGET = 'NONE'
SCOPETYPE = 'OPENADC'
SS_VER = 'SS_VER_1_1'
# DATA_LEN = 190
# RESP_LEN = 96

In [3]:
scope = cw.scope()
scope.default_setup()

scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 2582277                   to 24846152                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 29538459                 
scope.clock.adc_rate                     changed from 0.0                       to 29538459.0               
scope.clock.clkgen_

In [6]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../firmware/mcu/simpleserial-ascon
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CW308_STM32F3 with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
arm-none-eabi-gcc (15:10.3-2021.07-4) 10.3.1 20210621 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CW308_STM32F3 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
.
Compiling:
.
Compiling:
Compiling:
Compiling:
-en     simpleserial-ascon.c ...
Compiling:
-en     aead.c ...
.
-en     .././simpleserial/simpleserial.c ...
-en     .././hal//stm32f3/stm32f3_hal.c ...
-en     .././hal//stm32f3/stm32f3_hal_lowlevel.c ...
Compiling:
.
-en     .././hal//stm32f3/stm32f3_sysmem.c ...
Assembling: .././hal//stm32f3/stm32f3_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=ob

In [7]:
%run "Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍
scope.clock.adc_freq                     changed from 29538459                  to 31267009                 
scope.clock.adc_rate                     changed from 29538459.0                to 31267009.0               


In [8]:
fw_path = "../firmware/mcu/simpleserial-ascon/simpleserial-ascon-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13727 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13727 bytes


In [9]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

In [10]:
reboot_flush()

try:
    target.baud = BAUD_RATE
except:
    pass

print('Baudrate:')
print(target.baud)

# print('Available Commands:')
# print(*target.get_simpleserial_commands(),sep = "\n")

Baudrate:
38400


In [11]:
# import time
# target.flush()

# CMD_ECHO_TEST = 'e'

# data = bytearray.fromhex(''.join('{:02x}'.format(x) for x in list(range(DATA_LEN))))
# data1 = bytearray.fromhex(''.join('{:02x}'.format(x) for x in list(range(DATA_LEN))))
# data2 = bytearray.fromhex(''.join('{:02x}'.format(x) for x in list(range(DATA_LEN))))


# target.flush()
# target.simpleserial_write(CMD_ECHO_TEST, data)
# target.simpleserial_write(CMD_ECHO_TEST, data1)

# time.sleep(0.1)
# response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
# print(response)

# print(list(response))

In [13]:
import numpy as np
import time
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook
# from bokeh.resources import INLINE

# output_notebook(INLINE)

CMD_ASCON = 'a'

DATA_LEN = 190
RESP_LEN = 96

M = 0x01             
C = 0x02              
A = 0x04              
N = 0x08              
K = 0x10              
RUN_ENC = 0x20          
OMIT_RESULT = 0x80   

def pad(data):
    if len(data) > DATA_LEN:
        print("data length: " + str(len(data)))
    assert len(data) <= DATA_LEN
    if SS_VER == 'SS_VER_1_1':
        data += bytearray.fromhex('00' * (DATA_LEN - len(data)))
    return data

m = bytearray.fromhex('00000000000000000000000000000000') 
a = bytearray.fromhex('00')  
n = bytearray.fromhex('00000000000000000000000000000000')  
k = bytearray.fromhex('00000000000000000000000000000000') 


data = bytearray([N | K])
data += n  
data += k  
target.flush()
target.simpleserial_write(CMD_ASCON, pad(data))
time.sleep(0.1)

data = bytearray([M | A | RUN_ENC])
data += bytearray([len(m)]) + m  # Message
data += bytearray([len(a)]) + a  # Associated data

scope.arm()
target.flush()
target.simpleserial_write(CMD_ASCON, pad(data))
time.sleep(0.1)

ret = scope.capture()
if ret:
    print("Capture Timed Out!")
    reboot_flush()
else:
    trace = scope.get_last_trace()

response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
print(response)

mlen = len(m)
tlen = 40  # Updated tag length from 16 to 40 bytes

c = response[0:mlen]  
t = response[mlen:mlen + tlen] 

print('Ciphertext: ' + ''.join('{:02x}'.format(x) for x in c))
print('Tag: ' + ''.join('{:02x}'.format(x) for x in t))

print(trace)


bytearray(b'7ru1\x06\xa2\x91\x19+\x83G\xd8\x11\xd6\x93\xa5\xa8\xf7K\xcf\xf7c\xd7\xce\xe9C\xaf\x8a\xf5\\J0\x87\xb7\x87G\xd2e\xd6\xc1\xcdW\xcb\x07I\x97T,\xf5\xe8C%\xa5\x188"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
Ciphertext: 3772753106a291192b8347d811d693a5
Tag: a8f74bcff763d7cee943af8af55c4a3087b78747d265d6c1cd57cb074997542cf5e84325a5183822
[-0.0234375  -0.1953125  -0.15039062 ... -0.08007812  0.02246094
  0.07226562]


In [14]:
# import time
import logging
# import numpy as np
# import matplotlib.pyplot as plt
from tqdm.notebook import trange
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook
# from bokeh.models import Span
import numpy as np
import time
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook
# from bokeh.resources import INLINE

# output_notebook(INLINE)

CMD_ASCON1 = 'a'


# Flags
M1 = 0x01             
C1 = 0x02             
A1 = 0x04             
N1 = 0x08             
K1 = 0x10             
RUN_ENC1 = 0x20      
OMIT_RESULT1 = 0x80   


def pad(data):
    if len(data) > DATA_LEN:
        print("data length: " + str(len(data)))
    assert len(data) <= DATA_LEN
    if SS_VER == 'SS_VER_1_1':
        data += bytearray.fromhex('00' * (DATA_LEN - len(data)))
    return data

# Define input data
m1 = bytearray.fromhex('00000000000000000000000000000000')  
a1 = bytearray.fromhex('00')  
n1 = bytearray.fromhex('00000000000000000000000000000000')  
k1 = bytearray.fromhex('00000000000000000000000000000000')  

# Send nonce and key
data1 = bytearray([N1 | K1])
data1 += n1  
data1 += k1  
target.flush()
target.simpleserial_write(CMD_ASCON1, pad(data1))
time.sleep(0.1)

# Send plaintext and associated data
data1 = bytearray([M1 | A1 | RUN_ENC1])
data1 += bytearray([len(m1)]) + m1  
data1 += bytearray([len(a1)]) + a1  

scope.arm()
target.flush()
target.simpleserial_write(CMD_ASCON1, pad(data1))
time.sleep(0.1)

ret = scope.capture()
if ret:
    print("Capture Timed Out!")
    reboot_flush()
else:
    trace = scope.get_last_trace()

response1 = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
print(response1)

mlen1 = len(m1)
tlen1 = 40  
clen1 = mlen1 + tlen1  

c1 = response1[0:mlen1] 
t1 = response1[mlen1:mlen1 + tlen1]  

print('Ciphertext1: ' + ''.join('{:02x}'.format(x) for x in c1))
print('Tag1: ' + ''.join('{:02x}'.format(x) for x in t1))


bytearray(b'7ru1\x06\xa2\x91\x19+\x83G\xd8\x11\xd6\x93\xa5\xa8\xf7K\xcf\xf7c\xd7\xce\xe9C\xaf\x8a\xf5\\J0\x87\xb7\x87G\xd2e\xd6\xc1\xcdW\xcb\x07I\x97T,\xf5\xe8C%\xa5\x188"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
Ciphertext1: 3772753106a291192b8347d811d693a5
Tag1: a8f74bcff763d7cee943af8af55c4a3087b78747d265d6c1cd57cb074997542cf5e84325a5183822


In [15]:
results = []
traces = [] 

# output_notebook(INLINE)

CMD_ASCON11 = 'a'

# Flags
M11 = 0x01             
C11 = 0x02             
A11 = 0x04              
N11 = 0x08             
K11 = 0x10             
RUN_ENC11 = 0x20       
# RUN_DEC11 = 0x40     
OMIT_RESULT11 = 0x80   

def pad(data):
    if len(data) > DATA_LEN:
        print("data length: " + str(len(data)))
    assert len(data) <= DATA_LEN
    if SS_VER == 'SS_VER_1_1':
        data += bytearray.fromhex('00' * (DATA_LEN - len(data)))
    return data

m11 = bytearray.fromhex('00000000000000000000000000000000')  
a11 = bytearray.fromhex('00')  
n11 = bytearray.fromhex('00000000000000000000000000000000')  
k11 = bytearray.fromhex('00000000000000000000000000000000')  

data11 = bytearray([N11 | K11])
data11 += n11  
data11 += k11  
target.flush()
target.simpleserial_write(CMD_ASCON11, pad(data11))
time.sleep(0.1)

data11 = bytearray([M11 | A11 | RUN_ENC11])
data11 += bytearray([len(m11)]) + m11 
data11 += bytearray([len(a11)]) + a11 

scope.arm()
target.flush()
target.simpleserial_write(CMD_ASCON11, pad(data11))
time.sleep(0.1)

ret1 = scope.capture()
if ret1:
    print("Capture Timed Out!")
    reboot_flush()
else:
    trace = scope.get_last_trace()

response11 = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
print(response11)

# print the response
mlen11 = len(m11)
tlen11 = 40  # Updated tag length from 16 to 40 bytes

c11 = response11[0:mlen11]  
t11 = response11[mlen11:mlen11+tlen11]

print('Ciphertext11: ' + ''.join('{:02x}'.format(x) for x in c11))
print('Tag: ' + ''.join('{:02x}'. format(x) for x in t11))


bytearray(b'7ru1\x06\xa2\x91\x19+\x83G\xd8\x11\xd6\x93\xa5\xa8\xf7K\xcf\xf7c\xd7\xce\xe9C\xaf\x8a\xf5\\J0\x87\xb7\x87G\xd2e\xd6\xc1\xcdW\xcb\x07I\x97T,\xf5\xe8C%\xa5\x188"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
Ciphertext11: 3772753106a291192b8347d811d693a5
Tag: a8f74bcff763d7cee943af8af55c4a3087b78747d265d6c1cd57cb074997542cf5e84325a5183822


In [16]:
if scope._is_husky:
    scope.glitch.enabled = True
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
scope.io.hs2 = "glitch"
scope.glitch.width = 2
scope.glitch.offset = -4.0
scope.glitch.ext_offset=0
print(scope.glitch)
print(scope)

clk_src     = clkgen
mmcm_locked = True
width       = 1.953125
width_fine  = 0
offset      = -3.90625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 1
output      = clock_xor

cwlite Device
sn         = 50203220594a48303030323231323034
fw_version = 
    major = 0
    minor = 65
    debug = 0
gain = 
    mode = high
    gain = 30
    db   = 24.8359375
adc = 
    state          = False
    basic_mode     = rising_edge
    timeout        = 2
    offset         = 0
    presamples     = 0
    samples        = 5000
    decimate       = 1
    trig_count     = 1116
    fifo_fill_mode = normal
clock = 
    adc_src       = clkgen_x4
    adc_phase     = 0
    adc_freq      = 31267009
    adc_rate      = 31267009.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True

In [17]:
print(scope.clock)

adc_src       = clkgen_x4
adc_phase     = 0
adc_freq      = 31267009
adc_rate      = 31267009.0
adc_locked    = True
freq_ctr      = 0
freq_ctr_src  = extclk
clkgen_src    = system
extclk_freq   = 10000000
clkgen_mul    = 2
clkgen_div    = 26
clkgen_freq   = 7384615.384615385
clkgen_locked = True



In [18]:
scope.clock.adc_src = "clkgen_x1"
# scope.clock.adc_src = "clkgen_x4"
# scope.clock.freq = 10000000
# scope.clock.adc_src = "clkgen_x4"
# scope.clock.clkgen_freq = 10000000

scope.arm()
print(scope)

cwlite Device
sn         = 50203220594a48303030323231323034
fw_version = 
    major = 0
    minor = 65
    debug = 0
gain = 
    mode = high
    gain = 30
    db   = 24.8359375
adc = 
    state          = False
    basic_mode     = rising_edge
    timeout        = 2
    offset         = 0
    presamples     = 0
    samples        = 5000
    decimate       = 1
    trig_count     = 0
    fifo_fill_mode = normal
clock = 
    adc_src       = clkgen_x1
    adc_phase     = 0
    adc_freq      = 29538471
    adc_rate      = 29538471.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1         = serial_rx
    tio2         = serial_tx
    tio3         = high_z
    tio4         = high_z
    pdid         = high_z
    pdic         = high_z
    nrst

In [19]:
print(scope.clock)

adc_src       = clkgen_x1
adc_phase     = 0
adc_freq      = 29538471
adc_rate      = 29538471.0
adc_locked    = True
freq_ctr      = 0
freq_ctr_src  = extclk
clkgen_src    = system
extclk_freq   = 10000000
clkgen_mul    = 2
clkgen_div    = 26
clkgen_freq   = 7384615.384615385
clkgen_locked = True



In [22]:
import time
import logging
import numpy as np
# import matplotlib.pyplot as plt
from tqdm.notebook import trange
# from bokeh.plotting import output_notebook
      
CMD_ASCON111 = 'a'

M111 = 0x01              # data contains message 
C111 = 0x02              # data contains ciphertext
A111 = 0x04              # data contains associated data
N111 = 0x08              # data contains nonce
K111 = 0x10              # data contains key
RUN_ENC111 = 0x20        # run enc after data transmission
# RUN_DEC111 = 0x40        # run dec after data transmission
OMIT_RESULT111 = 0x80    # omit returning result after encryption/decryp

# output_notebook()

logging.getLogger().setLevel(logging.ERROR)

num_tests = 1000 
glitch_loc = range(121, 124) 
results = []
traces = []

for test in range(num_tests):
    reboot_flush()
    time.sleep(0.1)

    for i in trange(min(glitch_loc), max(glitch_loc) + 1, desc=f"Glitch Test {test+1}"):
        scope.adc.timeout = 0.2
        scope.glitch.ext_offset = i  
        scope.arm()

        
        m111 = bytearray.fromhex('00000000000000000000000000000000')  
        a111 = bytearray.fromhex('00')  
        n111 = bytearray.fromhex('00000000000000000000000000000000')  
        k111 = bytearray.fromhex('00000000000000000000000000000000')  

        
        data111 = bytearray([N111 | K111])
        data111 += n111  
        data111 += k111  
        target.flush()
        target.simpleserial_write(CMD_ASCON11, pad(data111))
        time.sleep(0.1)

        
        data111 = bytearray([M111 | A111 | RUN_ENC111])
        data111 += bytearray([len(m111)]) + m111  
        data111 += bytearray([len(a111)]) + a111  

        scope.arm()
        target.flush()
        target.simpleserial_write(CMD_ASCON111, pad(data111))
        time.sleep(0.1)

        ret1 = scope.capture()
        if ret1:
            print("⚠️ Capture Timed Out! Resetting...")
            reboot_flush()
            continue

        trace = scope.get_last_trace()

        
        glitched_response111 = target.simpleserial_read_witherrors('r', RESP_LEN, end='\n', timeout=250, ack=True)

        if glitched_response111['valid']:
            glitched_t111 = glitched_response111['payload'][mlen11:mlen11+tlen11]  

           
            if glitched_t111 != t11:
                print(f"\n Glitch at {i}:")
                print(f"   Original tag: {t11.hex()}") 
                print(f"   Glitched tag: {glitched_t111.hex()}")
                
                break  
        else:
            reboot_flush()


Glitch Test 1:   0%|          | 0/3 [00:00<?, ?it/s]

Glitch Test 2:   0%|          | 0/3 [00:00<?, ?it/s]

Glitch Test 3:   0%|          | 0/3 [00:00<?, ?it/s]

Glitch Test 4:   0%|          | 0/3 [00:00<?, ?it/s]


 Glitch at 123:
   Original tag: a8f74bcff763d7cee943af8af55c4a3087b78747d265d6c1cd57cb074997542cf5e84325a5183822
   Glitched tag: a8f74bcff56fd72de943af8ac15e480987b78747646bd03acd57cb07ff9952d7f5e84325a5183822


...

Glitch Test 998:   0%|          | 0/3 [00:00<?, ?it/s]

Glitch Test 999:   0%|          | 0/3 [00:00<?, ?it/s]


 Glitch at 123:
   Original tag: a8f74bcff763d7cee943af8af55c4a3087b78747d265d6c1cd57cb074997542cf5e84325a5183822
   Glitched tag: a8f74bcff56fd72de943af8ac15e480987b78747646bd03acd57cb07ff9952d7f5e84325a5183822


Glitch Test 1000:   0%|          | 0/3 [00:00<?, ?it/s]


 Glitch at 123:
   Original tag: a8f74bcff763d7cee943af8af55c4a3087b78747d265d6c1cd57cb074997542cf5e84325a5183822
   Glitched tag: a8f74bcff56fd72de943af8ac15e480987b78747646bd03acd57cb07ff9952d7f5e84325a5183822
